# YOLOv7 설치

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd /content/yolov7
!pip install -r requirements.txt

fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/yolov7


In [ ]:
# 데이터셋 저장될 폴더 생성
%mkdir /content/yolov7/moodata
%cd /content/yolov7/moodata
!pwd

/content/yolov7/moodata
/content/yolov7/moodata


In [ ]:
import yaml
import torch
from IPython.display import Image, clear_output

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터셋 확인 및 준비

In [ ]:
# 데이터셋 경로 설정
train_path = '/content/drive/MyDrive/moodata/train'
val_path = '/content/drive/MyDrive/moodata/val'

In [ ]:
from glob import glob

train_img_list = glob('/content/drive/MyDrive/moodata/train/images/*.jpg')
valid_img_list = glob('/content/drive/MyDrive/moodata/val/images/*.jpg')


print(len(train_img_list), len(valid_img_list))

9381 1120


# 데이터 전처리

Json 형식의 라벨링을 YOLOv7 annotation에 맞도록 변환하고 Class 수정

[Class] 0 : 정상 1 : 질병

In [ ]:
import json
import os
from tqdm import tqdm

def convert_annotation(json_file_path, output_file_path):
    """
    JSON annotation을 YOLO 형식으로 변환하여 저장하는 함수.

    Args:
        json_file_path (str): JSON 파일 경로.
        output_file_path (str): 변환된 annotation을 저장할 파일 경로.
    """
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at path: {json_file_path}")
        return  # 파일이 없으면 함수 종료

    # 이미지 크기 정보
    image_width = data["description"]["width"]
    image_height = data["description"]["height"]

    # Check if image_width or image_height is 0 and handle the error
    if image_width == 0 or image_height == 0:
        print(f"Error: Invalid image dimensions in {json_file_path}. Skipping this file.")
        return

    # disease 값 확인 및 라벨링 변환
    disease_value = data["annotations"]["disease"]
    label = 0 if disease_value == 0 else 1

    # 바운딩 박스 정보 가져오기
    points = data["annotations"]["points"][0]
    xtl = points["xtl"]
    ytl = points["ytl"]
    xbr = points["xbr"]
    ybr = points["ybr"]

    # YOLO 형식 변환 (중심 좌표 및 width, height 비율)
    x_center = (xtl + xbr) / 2 / image_width
    y_center = (ytl + ybr) / 2 / image_height
    bbox_width = (xbr - xtl) / image_width
    bbox_height = (ybr - ytl) / image_height

    # YOLO 형식의 텍스트 라인 생성
    yolo_annotation = f"{label} {x_center:.3f} {y_center:.3f} {bbox_width:.3f} {bbox_height:.3f}\n"

    # 텍스트 파일로 저장
    with open(output_file_path, 'w') as output_file:
        output_file.write(yolo_annotation)

# 데이터셋 경로 설정
train_path = '/content/drive/MyDrive/moodata/train'
val_path = '/content/drive/MyDrive/moodata/val'

# train/labels 및 val/labels 디렉토리의 JSON 파일 처리
for data_path, output_folder in [(train_path, 'train'), (val_path, 'val')]:
    labels_dir = os.path.join(data_path, 'labels')  # labels 폴더 경로
    output_dir = os.path.join(data_path, 'labels2')  # labels2 폴더 경로

    # labels2 폴더 생성
    os.makedirs(output_dir, exist_ok=True)

    # JSON 파일 목록 가져오기
    json_files = [f for f in os.listdir(labels_dir) if f.endswith('.json')]

    # 각 JSON 파일 처리
    for json_file in tqdm(json_files, desc=f"Processing {output_folder} data"):
        json_file_path = os.path.join(labels_dir, json_file)
        output_file_name = json_file.replace('.json', '.txt')
        output_file_path = os.path.join(output_dir, output_file_name)

        convert_annotation(json_file_path, output_file_path)

    print(f"{output_folder} 데이터셋의 변환된 파일이 '{output_dir}'에 저장되었습니다.")

Processing train data:   4%|▍         | 507/11698 [01:22<01:52, 99.70it/s] 

Error: Invalid image dimensions in /content/drive/MyDrive/moodata/train/labels/V006_79_1_03_02_03_12_3_3055e_20201117_12.json. Skipping this file.


Processing train data:   4%|▍         | 518/11698 [01:22<01:56, 95.83it/s]

Error: Invalid image dimensions in /content/drive/MyDrive/moodata/train/labels/V006_79_1_03_02_03_12_2_3055e_20201117_14.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/train/labels/V006_79_1_03_02_03_12_2_3055e_20201117_3.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/train/labels/V006_79_1_03_02_03_12_2_3055e_20201111_5.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/train/labels/V006_79_1_03_02_03_12_3_3055e_20201124_1.json. Skipping this file.


Processing train data:   5%|▍         | 552/11698 [01:22<01:47, 103.40it/s]

...

Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201124_16.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201117_6.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201124_7.json. Skipping this file.


Processing val data:  71%|███████   | 999/1413 [00:32<00:04, 90.62it/s]

Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201111_6.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201125_6.json. Skipping this file.
Error: Invalid image dimensions in /content/drive/MyDrive/moodata/val/labels/V006_79_1_03_02_03_12_2_3055e_20201104_6.json. Skipping this file.


Processing val data: 100%|██████████| 1413/1413 [00:37<00:00, 37.81it/s]

val 데이터셋의 변환된 파일이 '/content/drive/MyDrive/moodata/val/labels2'에 저장되었습니다.


# Yaml 수정

In [ ]:
# IPython에서 입력된 내용을 텍스트처럼 계속 사용하기 위한 register_line_cell_magic 정의
from IPython.core.magic import register_line_cell_magic

# register_line_cell_magic을 이용해 IPython에서 바로 사용할 수 있게 정의
# writeimplate를 쉽게 사용할 수 있음
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# yaml 파일 생성
data = dict(
    train =  '/content/drive/MyDrive/moodata/train',
    val = '/content/drive/MyDrive/moodata/val',
    nc = 2, # 클래스 개수
    names = ['normal', 'disease'] # 클래스 이름
)

with open('/content/yolov7/moodata/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
with open('/content/yolov7/moodata/data.yaml', 'r') as f:
    num_classes = str(yaml.safe_load(f)['nc'])
num_classes

'2'

# 층 정의(분류기 class 수 수정)

In [ ]:
!cat /content/yolov7/cfg/training/yolov7.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
  
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2      
   [-1, 1, Conv, [64, 3, 1]],
   
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4  
   [-1, 1, Conv, [64, 1, 1]],
   [-2, 1, Conv, [64, 1, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]],  # 11
         
   [-1, 1, MP, []],
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 16-P3/8  
   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],

In [ ]:
%%writetemplate /content/yolov7/cfg/training/yolov7.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0

   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Conv, [64, 3, 1]],

   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 1, Conv, [64, 1, 1]],
   [-2, 1, Conv, [64, 1, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]],  # 11

   [-1, 1, MP, []],
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 16-P3/8
   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1]],  # 24

   [-1, 1, MP, []],
   [-1, 1, Conv, [256, 1, 1]],
   [-3, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 29-P4/16
   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [1024, 1, 1]],  # 37

   [-1, 1, MP, []],
   [-1, 1, Conv, [512, 1, 1]],
   [-3, 1, Conv, [512, 1, 1]],
   [-1, 1, Conv, [512, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 42-P5/32
   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [1024, 1, 1]],  # 50
  ]

# yolov7 head
head:
  [[-1, 1, SPPCSPC, [512]], # 51

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [37, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],

   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]], # 63

   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [24, 1, Conv, [128, 1, 1]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],

   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1]], # 75

   [-1, 1, MP, []],
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3, 63], 1, Concat, [1]],

   [-1, 1, Conv, [256, 1, 1]],
   [-2, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]], # 88

   [-1, 1, MP, []],
   [-1, 1, Conv, [256, 1, 1]],
   [-3, 1, Conv, [256, 1, 1]],
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, -3, 51], 1, Concat, [1]],

   [-1, 1, Conv, [512, 1, 1]],
   [-2, 1, Conv, [512, 1, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1]], # 101

   [75, 1, RepConv, [256, 3, 1]],
   [88, 1, RepConv, [512, 3, 1]],
   [101, 1, RepConv, [1024, 3, 1]],

   [[102,103,104], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

In [ ]:
!cat /content/yolov7/cfg/training/yolov7.yaml


# parameters
nc: 2  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0

   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Conv, [64, 3, 1]],

   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 1, Conv, [64, 1, 1]],
   [-2, 1, Conv, [64, 1, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Conv, [64, 3, 1]],
   [[-1, -3, -5, -6], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1]],  # 11

   [-1, 1, MP, []],
   [-1, 1, Conv, [128, 1, 1]],
   [-3, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 2]],
   [[-1, -3], 1, Concat, [1]],  # 16-P3/8
   [-1, 1, Conv, [128, 1, 1]],
   [-2, 1, Conv, [128, 1, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 

In [ ]:
%cd /content/yolov7
!pwd

/content/yolov7
/content/yolov7


In [ ]:
!ls

cfg	detect.py  hubconf.py  models	README.md	  test.py	train.py
data	export.py  inference   moodata	requirements.txt  tools		utils
deploy	figure	   LICENSE.md  paper	scripts		  train_aux.py


# 학습

In [ ]:
!python train.py --img 320 --batch 16 --epochs 10  --data moodata/data.yaml --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --name yolov7_mooresult --cache

2024-10-15 00:10:11.772196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 00:10:11.794873: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 00:10:11.800928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 00:10:11.814802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 00:10:12.865371: W tensorflow/compiler/tf2

# 추론

원본 동영상 출처 링크 : https://www.youtube.com/shorts/9X99923ZnUg

In [ ]:
!python detect.py --weights /content/yolov7/runs/train/yolov7_mooresult/weights/best.pt --img 640 --conf 0.5 --source /content/drive/MyDrive/무밭.mp4

Namespace(weights=['/content/yolov7/runs/train/yolov7_mooresult/weights/best.pt'], source='/content/drive/MyDrive/무밭.mp4', img_size=640, conf_thres=0.5, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.4.1+cu121 CUDA:0 (Tesla T4, 15102.0625MB)

/content/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [27]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [30]:
import shutil
import os

source_dir = '/content/drive/MyDrive/yolov7_mooresult'
destination_dir = '/content/yolov7/runs/train/yolov7_mooresult'

# 대상 디렉토리가 없으면 생성
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# 소스 디렉토리의 모든 파일과 하위 디렉토리 복사
for item in os.listdir(source_dir):
    source_item_path = os.path.join(source_dir, item)
    destination_item_path = os.path.join(destination_dir, item)

    if os.path.isfile(source_item_path):
        shutil.copy2(source_item_path, destination_item_path)
    elif os.path.isdir(source_item_path):
        shutil.copytree(source_item_path, destination_item_path, dirs_exist_ok=True)

print("파일이 성공적으로 복사되었습니다.")


파일이 성공적으로 복사되었습니다.


원본동영상 출처 링크 : https://www.youtube.com/shorts/viTxBaHb2gU

# 결과 저장

In [ ]:
import os
import shutil
from google.colab import drive

# Google 드라이브 마운트
drive.mount('/content/drive')

# 원본 경로
source_path = '/content/yolov7/runs/train/yolov7_mooresult'

# 목적지 경로
destination_path = '/content/drive/MyDrive'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
